In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

/home/rootuser/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
import pandas as pd
import numpy as np

In [81]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
import re
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

In [5]:
df = pd.read_csv('data/train.tsv', sep = '\t')

In [10]:
df['length'] = df['Phrase'].apply(lambda x : len(str(x).split(' ')))

In [12]:
x = df['length']

In [13]:
max(x)

52

In [16]:
df = df.drop(columns = ['length','SentenceId','PhraseId'])

In [27]:
print(df[df.Sentiment == 1].size)

109092


In [28]:
print(df[df.Sentiment == 2].size)

318328


In [46]:
print(df[df.Sentiment == 3].size)

168532


In [44]:
print(df[df.Sentiment == 4].size)

0


In [47]:
print(df[df.Sentiment == 5].size)

0


In [ ]:
df.loc[df.Sentiment == 4, 'Sentiment'] = 3

In [90]:
df.loc[df.Sentiment == 0, 'Sentiment'] = 1

In [48]:
print(df[df.Sentiment == 4].size)

0


In [49]:
print(df[df.Sentiment == 3].size)

168532


In [22]:
df['Phrase'] = df['Phrase'].apply(lambda x: x.lower())

In [57]:
df['Phrase'] = df['Phrase'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]','',x))

In [59]:
max_features = 2000

In [62]:
tokenizer = Tokenizer(num_words=max_features, split=' ')

In [67]:
tokenizer.fit_on_texts(df['Phrase'].values)

In [70]:
X = tokenizer.texts_to_sequences(df['Phrase'].values)

In [76]:
X = pad_sequences(X, maxlen = 50)

In [82]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout = 0.2, recurrent_dropout = 0.2 ))
model.add(Dense(3, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [83]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 50, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [95]:
Y = pd.get_dummies(df['Sentiment']).values

In [99]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 222)

In [100]:
print(X_train.shape,Y_train.shape)

(124848, 50) (124848, 3)


In [101]:
print(X_test.shape,Y_test.shape)

(31212, 50) (31212, 3)


In [102]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 5, batch_size = batch_size, verbose = 2)

Epoch 1/5
 - 1101s - loss: 0.8123 - acc: 0.6543
Epoch 2/5
 - 858s - loss: 0.7395 - acc: 0.6933
Epoch 3/5
 - 824s - loss: 0.7127 - acc: 0.7061
Epoch 4/5
 - 812s - loss: 0.6928 - acc: 0.7145
Epoch 5/5
 - 825s - loss: 0.6789 - acc: 0.7221


In [103]:
from keras.models import model_from_json

In [104]:
model_json = model.to_json()
with open("model_first.json", "w") as json_file:
    json_file.write(model_json)

In [105]:
model.save_weights("model_first.h5")

In [106]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))

score: 0.71


In [107]:
json_file = open('model_first.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model_first.h5")

In [108]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
batch_size = 32
loaded_model.fit(X_train, Y_train, epochs = 5, batch_size = batch_size, verbose = 2)

Epoch 1/5
 - 815s - loss: 0.6659 - acc: 0.7271
Epoch 2/5


In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))